# Fetching crypto and tweets data.

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from crypto_api import CryptoApi
import pandas as pd
import numpy as np
import re


#btc_df = pd.read_csv(r'Data\btc_data.csv',usecols=lambda x: x != "Unnamed: 0")

# Analysis of tweets

In [2]:
'''
TODO: GENERAL DATA TASKS:
0) find a way to deal with multiple tweets for a day
1) merge 2 datasets into 
2) imput missing data, maybe try interpolation or expectation maximization
    2.1) compare with mean, median imput methods
3) ivestigate relationship within data, maybe correlation matrix etc
'''

tweets_df = pd.read_csv(r'Data/elon_tweets.csv', index_col=0)
tweets_df['date'] = pd.to_datetime(tweets_df['date'])
                                                
tweets_df = (tweets_df
             .dropna(axis=1, how='all')
             .drop(['vibe','cashtags'], axis=1)) # 1 and 18 notna values respectively  

## Dealing with sparse columns

In [3]:
sparse_cols = tweets_df.columns[tweets_df.notnull().mean() < 1.0].values.copy()

mod_tweets_df = tweets_df.copy()
mod_tweets_df = (mod_tweets_df[mod_tweets_df['lang']=='en']
                 .drop(['id','url','source','sourceUrl'], axis=1)                 
                 .reset_index(drop=True)
                 .copy())

mod_tweets_df = mod_tweets_df.drop(['lang'], axis=1)

## Data cleaning and preprocessing

In [4]:
encoder = OrdinalEncoder()
mod_tweets_df['sourceLabel_encoded'] = encoder.fit_transform(mod_tweets_df['sourceLabel'].values.reshape(-1, 1))
mod_tweets_df['isReplied']   = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['inReplyToUser']]
mod_tweets_df['isMentioned'] = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['mentionedUsers']]

#mod_tweets_df = mod_tweets_df.drop(['sourceLabel','inReplyToUser','mentionedUsers'], axis=1)


def extract_dict(line: str, prepare_to_df: False):
    """Extracts data from a dict represented as string and makes it a dict.

    ## Parameters:
        line (str): row of a Series/DataFrame to be preprocessed.
        prepare_to_df (bool): prepares extracted dict to be wrapped into DataFrame.

    ## Returns:
        dict: extracted dict from string.
    """    

    extracted_content = dict(re.findall(r"'(\w+)': '?({.*}|datetime.datetime\(.*\)|[\w\d/:\. ]*)'?", line))
    
    # Wraps dict values into lists to be easily represented as a DataFrame row.
    if prepare_to_df:
        for key,value in extracted_content.items():
            if value == '':
                extracted_content[key] = [None]
            else:
                extracted_content[key] = value
        
    return extracted_content


new_df = mod_tweets_df.copy()     
extracted_df = (pd.DataFrame([*mod_tweets_df['user']
                              .apply(lambda x: extract_dict(x, True))])
                )

new_df = (pd.concat([new_df, extracted_df], axis=1)
            .drop(['user','username','id','displayname','verified','created',
                    'location','protected','profileImageUrl','profileBannerUrl',
                    'rawDescription','renderedDescription','favouritesCount',
                    'friendsCount','mediaCount','statusesCount'], axis=1))

In [5]:
# Converting columns containing numbers to int after extraction.
for column in new_df:
    if 'Count' in column:
        new_df[column] = new_df[column].astype('Int64').copy()

In [6]:
new_df[['rawContent','isReplied','isMentioned']].query("rawContent.str.contains('@')")

new_df['mentionsCount'] = new_df['rawContent'].str.count(r'@[\w\d]+')
new_df['mentions'] = new_df['rawContent'].apply(lambda x : re.findall(r'(@[^\s]+)', x))

count = 0
for a,b in new_df[['mentionsCount','mentions']].values:
    if a==len(b):
        count +=1 
print(count==len(new_df))

new_df['charCount'] = new_df['rawContent'].apply(lambda x: len(x))
new_df = new_df.drop('descriptionLinks', axis=1)

True


In [7]:
#links = tweets_df[tweets_df.columns[tweets_df.columns.isin(new_df.columns)==False]]['links'].value_counts().copy()

tweets_df[tweets_df['rawContent']=='True']['quotedTweet'].iloc[0]

'{\'url\': \'https://twitter.com/ggreenwald/status/1625871270737809408\', \'date\': datetime.datetime(2023, 2, 15, 14, 54, 52, tzinfo=datetime.timezone.utc), \'rawContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'renderedContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'id\': 1625871270737809408, \'user\': {\'username\': \'ggreenwald\', \'id\': 16076032, \'displayname\': \'Glenn Greenwald\', \'rawDescription\': \'Journalist; Author; Host, @SystemUpdate_; Columnist, @Folha; Co-Founder: The Intercept, @TheInterceptBr; @abrigo_hope, @FreedomofPress, @ongcriadefavela.

# Model Bulding

## Training pipeline 

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, matutils, utils, models
from gensim.models import ldamodel, ldamulticore
import spacy


def lemmatization(nlp, texts:list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    if type(texts)!=list:
        texts = [texts]
    
    for text in texts:
        # TODO: consider using nlp.pipe which should be faster
        doc = nlp(text)
        cleaned_text = []
        for token in doc:
            if token.ent_type_ != 'URL' and not token.is_stop and token.pos_ in allowed_postags:
                cleaned_text.append(token.lemma_)
        final = ' '.join(cleaned_text)
        texts_out.append(final)

    return texts_out


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
url_pattern = [{"label": "URL",
                "pattern": [{"LIKE_URL": True}]}]

ruler = nlp.add_pipe('entity_ruler', before='ner')
ruler.add_patterns(url_pattern)


# lemmatized_texts = lemmatization(nlp, new_df['rawContent'])
with open('lemmatized_texts.txt', 'r',encoding="utf-8") as f:
    lemmatized_texts = f.readlines()
    
lemmatized_texts = [line.replace('\n','') for line in lemmatized_texts]

In [76]:
def create_ngrams(texts):
    data_words = []
    for text in texts:
        new = utils.simple_preprocess(text)
        data_words.append(new)

    bigrams_phrases  = models.Phrases(data_words, min_count=3, threshold=50)
    trigrams_phrases = models.Phrases(bigrams_phrases[data_words], threshold=50)

    bigram  = models.phrases.Phraser(bigrams_phrases)
    trigram = models.phrases.Phraser(trigrams_phrases)

    data_bigrams = [bigram[doc] for doc in data_words]
    data_bigrams_trigrams = [trigram[bigram[doc]] for doc in data_bigrams]
    
    return data_bigrams_trigrams


data_bigrams_trigrams = create_ngrams(lemmatized_texts)

In [77]:
def vectorize_texts(texts_ngrams):
    id2word = corpora.Dictionary(texts_ngrams)
    corpus = [id2word.doc2bow(text) for text in texts_ngrams]

    return id2word, corpus

id2word, corpus = vectorize_texts(data_bigrams_trigrams)

#tfidf = models.TfidfModel(corpus, id2word=id2word)
# tfidf_vectorizer = TfidfVectorizer(max_df=0.6,
#                                    min_df=5,
#                                    ngram_range=(1,3))
# tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_texts)

# id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())
# corpus = matutils.Sparse2Corpus(tfidf_matrix.T)

In [12]:
# low_value = 0.03
# words = []
# words_missing_in_tfidf = []

# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id,_ in tfidf[bow]]
#     bow_idf = [id for id,_ in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
    
#     for item in drops:
#         words.append(id2word[item])
    
#     # words with tfidf score of 0 will be missing
#     words_missing_in_tfidf = [id for id in bow_idf if id not in tfidf_ids]
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

In [13]:
# id2word = corpora.Dictionary(data_words)
# corpus = [id2word.doc2bow(text) for text in data_words]

In [14]:
from gensim.models import CoherenceModel, LdaMulticore


def bayesian_tuning(model, params_grid: dict, texts, verbose=False):
    # TODO: implement bayesian tuning
    models_scores = {}
    for i in range(95,170,5):
        lda_model = model(corpus=params_grid['corpus'],
                          num_topics=i,
                          id2word=params_grid['id2word'],
                          random_state=1,
                          passes=10,
                          per_word_topics=True)
        
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                             texts=texts, 
                                             corpus=params_grid['corpus'], 
                                             dictionary=params_grid['id2word']
                                             )
        coherence_score = coherence_model_lda.get_coherence()
        
        models_scores.update({lda_model: coherence_score})
        
        if verbose:
            print(f'Topics {i:<3}: {coherence_score}')
    
    return models_scores


params_grid = {'corpus':corpus,  
               'num_topics':10, 
               'id2word':id2word, 
               'random_state':1, 
               'update_every':1, 
               'chunksize':3000, 
               'passes':2}

# lda_models_scores = bayesian_tuning(LdaMulticore, params_grid, texts, verbose=True)

In [17]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()

# vis = gensimvis.prepare(best_model, corpus, id2word)
# vis

lda_model = LdaMulticore(corpus=corpus,
                         num_topics=25,
                         id2word=id2word,
                         random_state=1,
                         passes=10,
                         per_word_topics=True)

vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
20    -0.058482 -0.093963       1        1  5.819648
24    -0.081258 -0.104785       2        1  5.475173
7     -0.118493 -0.035073       3        1  5.036833
19     0.061898 -0.076325       4        1  4.475814
8     -0.020586 -0.009388       5        1  4.287076
17     0.157020  0.059253       6        1  4.279670
9     -0.038438 -0.096719       7        1  4.261470
4     -0.057585  0.085894       8        1  4.227003
0     -0.020299 -0.053640       9        1  4.224180
14    -0.062728  0.037820      10        1  4.199254
21    -0.100026  0.020621      11        1  4.096095
23     0.018872  0.010184      12        1  4.093045
1     -0.029935 -0.036383      13        1  4.031689
15    -0.117480  0.151056      14        1  3.935348
22     0.030813  0.065815      15        1  3.829052
11     0.047996  0.024274      16        1  3.616430
5      0.106233  0.045819      17        1  3.595053
18    -0.017964  0.005432      18        1  3.592808
10     0.015147 -0.059452      19        1  3.413737
3     -0.002409 -0.047485      20        1  3.339997
13    -0.006572  0.164138      21        1  3.323793
2      0.065431 -0.002421      22        1  3.319342
16     0.122976 -0.058457      23        1  3.313656
12     0.041694  0.039725      24        1  3.196872
6      0.064173 -0.035939      25        1  3.016960, topic_info=            Term        Freq       Total Category  logprob  loglift
29          true  325.000000  325.000000  Default  30.0000  30.0000
999      exactly  346.000000  346.000000  Default  29.0000  29.0000
9232      billym  445.000000  445.000000  Default  28.0000  28.0000
70          good  787.000000  787.000000  Default  27.0000  27.0000
32         great  553.000000  553.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
10193  stillgray   10.308624  112.778632  Topic25  -5.7212   1.1085
1265     company   10.883116  203.747674  Topic25  -5.6670   0.5713
207        thing   10.798456  285.661232  Topic25  -5.6748   0.2255
0       actually   10.146979  265.877117  Topic25  -5.7370   0.2351
669         high    9.985973  405.371144  Topic25  -5.7530  -0.2027

[1838 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
9639      22  0.790129      abide
1353       1  0.156873       able
1353       2  0.028522       able
1353       4  0.085567       able
1353       5  0.057045       able
...      ...       ...        ...
10753      2  0.844923  zerohedge
10753     11  0.105615  zerohedge
2589       2  0.086895       zone
2589      10  0.782051       zone
10885     16  0.731589  zubymusic

[6890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 25, 8, 20, 9, 18, 10, 5, 1, 15, 22, 24, 2, 16, 23, 12, 6, 19, 11, 4, 14, 3, 17, 13, 7])

In [33]:
lda_model.get_term_topics('tesla')

[(21, 0.027174275)]

In [35]:
btc_df

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2017-11-15,7330.06,6596.94,6597.06,131120.23,9.229595e+08,7283.22,direct,NaN
1,2017-11-16,7964.64,7119.17,7283.02,133937.80,1.010131e+09,7853.68,direct,NaN
2,2017-11-17,8000.19,7534.70,7853.68,117347.03,9.175999e+08,7699.95,direct,NaN
3,2017-11-18,7857.52,7458.90,7699.95,74382.12,5.711211e+08,7780.91,direct,NaN
4,2017-11-19,8100.87,7675.41,7781.02,68618.85,5.413913e+08,8042.64,direct,NaN
...,...,...,...,...,...,...,...,...,...
1996,2023-05-04,29370.41,28703.83,29041.00,27555.47,7.986150e+08,28866.54,direct,NaN
1997,2023-05-05,29697.23,28853.10,28866.54,35357.40,1.037025e+09,29550.84,direct,NaN
1998,2023-05-06,29857.86,28459.77,29550.84,24511.97,7.131925e+08,28943.94,direct,NaN
1999,2023-05-07,29207.72,28547.14,28943.94,16591.33,4.802422e+08,28574.43,direct,NaN


In [70]:
puppy_text = 'Rocket launch is tommorow, hope for good landing, satelite awaits for us.'
lemmatized_puppy = lemmatization(nlp, puppy_text)
words = gen_words(lemmatized_puppy)

#lda_model.get_document_topics(puppy)
words

[['rocket',
  'launch',
  'tommorow',
  'hope',
  'good',
  'landing',
  'satelite',
  'await']]

In [82]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

preprocessing_pipeline = Pipeline([
    ('lemmatization', FunctionTransformer(lemmatization, kw_args={'nlp': nlp})),
    ('trigrams', FunctionTransformer(create_ngrams)),
    ('vectorization', TfidfVectorizer())
    ])

In [83]:
preprocessing_pipeline.transform(puppy_text)

TypeError: lemmatization() got multiple values for argument 'nlp'